!jupyter nbconvert index.ipynb --to slides --reveal-prefix reveal.js --post serve

<center>MINISTRY OF EDUCATION AND SCIENCE OF UKRAINE<br>
NATIONAL TECHNICAL UNIVERSITY OF UKRAINE<br>
"IGOR SIKORSKY KYIV POLYTECHNIC INSTITUTE"<br>
INSTITUTE OF APPLIED SYSTEM ANALYSIS<br>
DEPARTMENT OF MATHEMATICAL METHODS OF SYSTEM ANALYSIS</center>


### Thesis on:
# Application of Long short-term memory algorythm in human activity recognition problems
<br>

<div style="text-align: right">
Done by:<br>
Valentyn Melnychuk, КА-41<br>
Supervisor:<br>
Ph.D., Associate Professor, Alla Yakovleva <br>
</div>

## 1. Introduction

### Object of study
A deep learning algorithm - **long short-term memory (LSTM)** for classification of human activity.


### Subject of study
Application of the LSTM in three classification tasks:
- classification of human activity by video (handclapping, handwaving, jogging, running, etc.)
- classification of human activity by data from gyroscope and accelerometer (walking, walking upstairs, walking downstairs, etc.)
- recognition of reading by the human gaze trajectory

### Aim of the study
- verification of the efficiency of the LSTM algorithm in tasks of activity classification
- study of its modifications and combination with other methods of deep learning
- stating advantages and disadvantages of the algorithm

## 2. Formulation of the problem

The classification model for sequence data is described as follows:
<center>Input sequence $\{x^{(t)} \in \mathbb{R}^n, t \in (1,2 ... \tau)\}$ -> Category from some fixed set $\hat{y} \in (1,2 ... K) $<center/>
<br>
    
One constructs a model that will estimate the dependence according to the given **quality criterion** based on the given sample:    
    
$$D = \Big\{(x_i, y_i); x_i = \{x^{(t)}_i \in \mathbb{R}^n, t \in (1,2 ... \tau_i)\}, \\ y_i \in (1,2 ... K), i \in (1, ..., m)\Big\}$$



## 3.1. KTH dataset

KTH dataset was created by Schuldt et. al. in 2004 and is the most popular dataset for human activity recognition.

Input data: 
- 600 videos of different length (100 for each category)
- Video is a sequence of monochrome frames (matrices) of size 120*160 

Output data:
- 6 types of activity: 'boxing', 'handclapping', 'handwaving', 'jogging', 'running', 'walking'
<br>
<img src="actions.jpg" width="700">
<center>Image 1. 6 types of activity</center>

## 3.2. HAR dataset

The HAR dataset was built from records of 30 participants, which performed 6 daily activities by holding a smartphone on waist with integrated accelerometer and gyroscope

Input data:
- 7352 time series of length 128 time steps
- Each time series - sequence of 9-dimensional vactors, which contain information about the acceleration and angular velocity

Output data:

- 6 types of activity: 'walking', 'walking upstairs', 'walking downstairs', 'sitting', 'standing', 'laying'
<br>
<img src="har_sample.png" width="600">
<center>Image 2. Accelaration in 3 dimentions for 4 types of activity</center>

## 3.3. GAZEPOINT dataset

One used dataset generated by the Gazepoint gaze tracker. 15 participants took part in the dataset recording, each of whom had to read two texts, find a specific picture, search in the text and watch a video.

Input data:
- 143 sequences with the trajectory of human gaze, recorded with a frequency of 60 Hz 
- each sequence has following features: gaze coordinates on screen and duration of a fixation.

Output data:

- 2 types of activity: 'reading', 'non-reading'
<br>
<img src="gazepoint_sample_reading.png" width="600">
<center>Image 3. Trajectory of gaze movement and duration of fixation for reading and non-reading</center>

## 4. Quality criterion for model selection

As quality criterions $J(D, \theta)$ for model with parameters $\theta$ were chosen the following metrics:
- average value of cost function  $L$ on validation subset: 
$$J (D, \theta ) = \frac{1}{m_{valid}} \sum_{i=1}^{m_{valid}}{L(x_i, y_i)}$$
- classification accuracy on validation subset:
$$J (D, \theta ) = \frac{1}{m_{valid}} \sum_{i=1}^{m_{valid}}{\mathbb{I}_{y_i = \tilde{y}_i}}$$
- confusion matrix on validation subset:
$$J (D, \theta ) =  {\vert\vert \sum_{i=1}^{m_{valid}}{\mathbb{I}_{y_i = k, \tilde{y}_j = l}} \vert\vert }_{k, l \in (1, ..., K)}$$

## 5. Overview of existing approaches to problem solving

- Classical activity recognition approaches require **feature engineering**, since traditional **feed-forward neural networks** can not handle sequences of different lengths and do not have the **memory property**.

- Recurent Neural Networks (RNNs), that contain **feedback-loop**, help to solve these disadvantages and allow to store information about the whole sequence of input data. The main problem of RNN - **vanishing gradient problem** and the usual RNN does not take into account long-term dependencies.

- The LSTM partially solves this problem, since it recursively transfers the state without applying activation or multiplying it by the matrix of weights, using gates.

## 6.1. Recurent neural networks
**Recurent neural network** - neural network, which implements the idea of **parameters sharing**, and is designed to work with sequences of arbitrary lengths $x^{(1)}, ..., x^{(\tau)}$. In fact, it is nonlinear autoregressive exogenous model (NARX).

Computational graph of vanilla RNN in unfolded view | Model equation 
- | - 
<img src="rnn.png" style="width: 350px;"> | <img src="rnn_formula.png" style="width: 300px;">

## 6.2. Types of Recurent neural networks

There are three main architectures of the RNNs, depending on the task: 

Many-to-many | Many-to-one | One-to-many 
- | - 
<img src="rnn.png" style="width: 300px;"> | <img src="many_to_one.png" style="width: 220px;"> | <img src="one-to-many.png" style="width: 270px;">
 

## 6.3. Recurent neural networks in classification problems

The most commonly used architecture is **many-to-one**, sometimes **many-to-many** (continious classification).

Prediction of **many-to-one** model will be: $$\hat{y} = softmax(o); \quad softmax(z)_{i}={\frac {e^{z_{i}}}{\sum _{k=1}^{K}e^{z_{k}}}}, i = 1, ..., K$$

Loss function $L$ for pair $(x, y)$ of input sequence and output category will be cross-entropy (negative log-likelihood):

$$L(\{x^{(1)}, ..., x^{(\tau)}\}, y \}) = - \sum_i{(y)_i * ln ((\hat{y})_i)}$$
where $\hat{y}$ - predicted output value, $y$ - real output value in one-hot encoding.

## 6.4. Recurent neural networks in classification problems

To find the weights of the model it is necessary to minimize the cost function:
$$ J = \frac{1}{m_{train}} \sum_{i=1}^{m_{train}}{L(x_i, y_i)}$$

Dependencies in RNN between $\hat{y}$ and $\{x^{(1)}, ..., x^{(\tau)}\}$ is described by differentiatable functions, thus we can use gradient-based methods.

The **Adam optimizer**, which is a modified stochastic gradient descent, is used in the work.

## 7.1. Long short-term memory

Long short-term memory (LSTM) is the architecture of recurrent neural networks, proposed in 1997 by Zepp Hohreiter and Jürgen Schmidguber.

The LSTM is specifically designed to avoid **vanishing gradient problem**, which is inherent in conventional RNNs.

Vanilla LSTM is described by following equations:
$$ \begin{split}
    f^{(t)}  & = \sigma(W_f [h^{(t-1)}, x^{(t)}] + b_f)\\
    i^{(t)} & = \sigma(W_i  [h^{(t-1)}, x^{(t)}] + b_i) \\
 C^{(t)} & = f^{(t)} \circ C^{(t-1)} + i^{(t)} \circ tanh(W_C  [h^{(t-1)}, x^{(t)}] + b_C) \\
 \tilde{h}^{(t)} & = \sigma(W_o  [h^{(t-1)}, x^{(t)}] + b_o) \\
  h^{(t)} & = \tilde{h}^{(t)} \circ tanh(C ^{ {(t)}})
 \end{split}
$$

## 7.2. Computational graph of LSTM

LSTM contains 4 recurrent layers and 3 gates: input, forgetting and output. 

<img src="lstm_new1.png" width="600">
where rectangles are layers of network, $\circ$ - Hadamard product.

 



## 8.1. KTH - results

Data pre-processing:
- Differentiation $x'^{(t)} = x^{(t)} - x^{(t-1)}$, to take into account only moving parts of frames and normalize the data
- Cropping on subsequences of 25 frames

<img src="kth_difference.png" width="500">

Two models were developed:
- CNN-LSTM - 3-dimensional CNN + 2-layers LSTM
- CONV-LSTM - convolutional LSTM (3 layers)

Results  |Confusion matrix of the best model
- | - 
<img src="kth_result.png" style="width: 450px;"> | <img src="kth_confusion.png" alt="Drawing" style="width: 300px;">

<video src="kth_test.mp4" width="600" controls>

## 8.2. HAR - results

Without data pre-processing.

3 models were developed:
- vanilla LSTM 
- LSTM with $L_2$ regularization
- LSTM with dropout

Results | Confusion matrix of the best model
- | - 
<img src="har_result.png" style="width: 450px;"> | <img src="har_confusion.png" style="width: 300px;">

## 8.3. GAZEPOINT - results

Data pre-processing:
- Differentiation of the trajectory of a gaze $x'^{(t)} = x^{(t)} - x^{(t-1)}$.

4 models were developed:
- Vanilla LSTM with 3-dimensional input sequence, 100 time steps
- LSTM-reshape(6) with 6-dimensional input sequence, 50 time steps
- LSTM-reshape(12) with 12-dimensional input sequence, 25 time steps
- CNN-5 - 5-layers convolutional neural network (for comparison)

<img src="gp_result.png" style="width: 450px;">

## 9. Dependency of the quality of the model on the length of the input sequence
 
KTH  | HAR 
- | - 
<img src="kth_different.png" alt="Drawing" style="width: 450px;"> | <img src="har_different.png" alt="Drawing" style="width: 450px;">


## 10. Further ideas for research

- Training on sequences of different lengths (one must have a priori knowledge of the duration of the dependency).
- Comparison of various LSTM architectures (peephole, with combined input and forgetting gates). 

## 11.1. Conclusions - advantages

In this paper, research was conducted on the possibility and effectiveness of the use of the LSTMs for the problems of classification of human activity.

Among the advantages of the LSTMs, one can mention the following:
- No need in pre-processing
- Scalability and universatility
- Ability to work with different lengths of input sequences

## 11.2. Conclusions - disadvantages

In addition to the advantages of method listed above, one revealed some of its disadvantages:

- The LSTM does not really require features engineering, but in some cases it is more accurate to build a model on input with pre-processing
- The LSTM is not a universal algorithm for sequences; for example, it could not properly classify 3-dimensional sequences of GAZEPOINT dataset
- It may sometimes be difficult to pick up the optimal model architecture
- Fitting these models may take some time

# Thanks for the attetion!